## Chains
- Chains refer to a sequence of calls - whether to an LLM, a tool, or a data preprocessing step. In LangChain, the primary supported way to do this is with LCEL (LangChain Expression Language).
- It is basically a pipeline where the stages can be either a call to an LLM or some other processing step.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY', '')

In [3]:
# Run this cell if you want to make your display wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
# Initialize the model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=openai_api_key)

### Simple LLM Chains
- Chains can be simple (i.e. Generic) or specialized (i.e. Utility)
- A simple LLM Chain relies on the prompt to respond.

In [5]:
# A Simple LLM Chain
llm.invoke("Tell me a Joke")

AIMessage(content="Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!")

### Prompt Template
- Prompt Template in langchain can be used to create complex prompts that contain variables and various other actions.
- Prompt templates are used to convert raw user input to a better input to the LLM.

In [6]:
# LLM Chain using a prompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template="Tell me a joke about {random_obj}")

We can combine the LLM model and the prompt template into a simple LLM chain  
chain = prompt | llm

In [7]:
chain = prompt| llm

In [8]:
# Now invoke the chain to get the result
chain.invoke({"random_obj": "cat"})

AIMessage(content='Why was the cat sitting on the computer?\nBecause it wanted to keep an eye on the mouse!')

The Prompt Template can also be used to add a system prompt for the model or to assign personas to the LLM responses.

In [9]:
msg_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are John Oliver, a late night talk show host."),
    ("user", "{input}")
])

- This can be combined with an output parser that returns the output in a form that is more convenient to work with. For ex in this case it will be a text output instead of an AIMessage object.  
- The chain will be of the type - prompt | llm | output_parser

In [10]:
# Add an output parser to the LLM chain
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [11]:
# Add this parser to the previous chain
chain = msg_prompt | llm | output_parser

We can now invoke this chain and ask it a question. The answer will now be a string (rather than a ChatMessage)

In [14]:
chain.invoke({"input": "Tell me about China"})

"China is a vast and diverse country located in East Asia, with a population of over 1.4 billion people, making it the most populous country in the world. It has a rich history spanning thousands of years, with a unique culture that has influenced much of the world.\n\nChina is known for its iconic landmarks such as the Great Wall, the Forbidden City, and the Terracotta Army. It is also home to a variety of landscapes, from bustling cities like Beijing and Shanghai to beautiful natural wonders like the Yangtze River and the karst mountains of Guilin.\n\nEconomically, China is one of the world's largest economies and a major player in global trade. It is known for its manufacturing industry, producing a wide range of goods from electronics to textiles. In recent years, China has also made significant investments in technology and innovation, leading to the rise of companies like Alibaba, Tencent, and Huawei.\n\nPolitically, China is governed by the Communist Party of China, with Preside

**Deep Dive**  
- The above section covers the basics of prompts, models and output parsers. 
- For a deep dive on these topics refer: https://python.langchain.com/docs/modules/model_io

### Retrieval Chain
- The LLMs that we use such as OpenAI or any open source model are trained on generic data, i.e. data from internet. So if we ask it a question that is specific to our use case, we need to provide additional context to the LLM.
- This can be done using **retrieval**. Retrieval is useful when we have too much data to pass to the LLM directly. Each LLM has a limit of tokens that can be passed in one API call.
- We can then use the retriever to fetch only the most relevant pieces and pass those in.
- In this process we look up relevant documents from a Retriever and then pass them into the prompt. A retriever can be backed by anything - a SQL Table, the internet etc.  
**Source** - https://python.langchain.com/docs/get_started/quickstart#retrieval-chain

In the example below we will populate a vector store and use that as a retriever. For more details on **vector stores** refer https://python.langchain.com/docs/modules/data_connection/vectorstores

In [17]:
# Load the data to index
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [18]:
docs

[Document(page_content="\n\n\n\n\nLangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Set

In [19]:
# Open AI Embeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

Now we can use the embeddings and the text data to ingest documents into a vector store. Here we are using a local vector store for simplicity.

In [22]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain

In [21]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

- Now that the data is indexed in the vector store, we will create a retrieval chain. 
- This will take an incoming question, look up relevant documents and then pass these documents along with the original question into an LLM and ask it to answer the original question.

In [23]:
# create a prompt and document chain
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context

<context>
{context}
</context>
Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [24]:
# Create a retrieval chain
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [25]:
# Now invoke this chain 
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by allowing users to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It provides tracing and evaluation capabilities to assist with testing the applications.


#### Diving Deeper
- The above example displays only the basics of retrieval chain. For aa deeper dive into it see -  **https://python.langchain.com/docs/modules/data_connection**

### Conversation Retrieval Chain
- The chain created in the Retrieval Chain above can only answer single questions. To convert the simple retrieval chain to answer follow up questions - i.e. chatbot we need to change 2 things:
    1. The retrieval method should now not just work on the most recent input but take the whole history into account.
    2. The final LLM chain should likewise take the whole history into account.
- **Updating Retrieval** 
    - In order to update retrieval, we will create a new chain.
    - This chain will take the most recent input and the chat history and use an LLM to generate a search query 

In [28]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [31]:
# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
aware_retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [32]:
# Create chat history
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
aware_retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following sections to learn more about LangSmith:User Guide: Learn about the workflows LangSmith supports at each stage of the LLM application lifecycle.Setup: Learn 

With this new retriever, we can create a new chain to continue the conversation with these documents

In [34]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, final_prompt)

final_retrieval_chain = create_retrieval_chain(aware_retriever_chain, document_chain)

Test this End to End

In [35]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
final_retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\nSkip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationMonitoringPrompt HubLangSmithOn this pageLangSmithIntroduction\u200bLangSmith is a platform for building production-grade LLM applications.It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.LangSmith is developed by LangChain, the company behind the open source LangChain framework.Quick Start\u200bTracing: Get started with the tracing quick start.Evaluation: Get started with the evaluation quick start.Next Steps\u200bCheck out the following s